In [ ]:
# Import libraries

import ssl
ssl._create_default_https_context = ssl._create_unverified_context

import numpy as np
import pandas as pd
import tensorflow as tf


In [ ]:
# Data preparation
data = pd.read_csv('data.csv')
data.head()

# URL
# URL = "https://raw.githubusercontent.com/aviralb13/codes/main/datas/Churn_Modelling.csv"
# data = pd.read_csv(URL)

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [ ]:
# Defining variables
x = data.iloc[:, 3:-1].values
y = data.iloc[:, -1].values
x

array([[619, 'France', 'Female', ..., 1, 1, 101348.88],
       [608, 'Spain', 'Female', ..., 0, 1, 112542.58],
       [502, 'France', 'Female', ..., 1, 0, 113931.57],
       ...,
       [709, 'France', 'Female', ..., 0, 1, 42085.58],
       [772, 'Germany', 'Male', ..., 1, 0, 92888.52],
       [792, 'France', 'Female', ..., 1, 0, 38190.78]], dtype=object)

In [ ]:
y

array([1, 0, 1, ..., 1, 1, 0])

In [ ]:
# Label encoding (Gender)
from sklearn.preprocessing import LabelEncoder
data = LabelEncoder()
x[:, 2] = data.fit_transform(x[:, 2])
print(x)

[[619 'France' 0 ... 1 1 101348.88]
 [608 'Spain' 0 ... 0 1 112542.58]
 [502 'France' 0 ... 1 0 113931.57]
 ...
 [709 'France' 0 ... 0 1 42085.58]
 [772 'Germany' 1 ... 1 0 92888.52]
 [792 'France' 0 ... 1 0 38190.78]]


In [ ]:
# One hot encoding (Geography)
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
column = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [1])], remainder='passthrough')
x = np.array(column.fit_transform(x))
print(x)

[[1.0 0.0 0.0 ... 1 1 101348.88]
 [0.0 0.0 1.0 ... 0 1 112542.58]
 [1.0 0.0 0.0 ... 1 0 113931.57]
 ...
 [1.0 0.0 0.0 ... 0 1 42085.58]
 [0.0 1.0 0.0 ... 1 0 92888.52]
 [1.0 0.0 0.0 ... 1 0 38190.78]]


In [ ]:
# Prepping the dataset

# Splitting the dataset
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y)

In [ ]:
# Feature scaling - scale down high values to normalise values
# A particular value in a dataset is very high, it starts dominating the model, reducingg the model accuracy

from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
x_train = sc.fit_transform(x_train)
x_test = sc.transform(x_test)

In [ ]:
# Setting up the model
# Steps:
# 1. Start a sequential model with tensorflow keras
# 2. Add hidden layers to the model (3 in this case)
# 3. Add one layer to compile above layers

ann = tf.keras.models.Sequential()
ann.add(tf.keras.layers.Dense(units=6, activation='relu'))
ann.add(tf.keras.layers.Dense(units=6, activation='relu'))
ann.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))
ann.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
ann.fit(x_train, y_train, batch_size=32, epochs=100)

Epoch 1/100
235/235 [==============================] - 2s 2ms/step - loss: 0.7543 - accuracy: 0.5277
Epoch 2/100
235/235 [==============================] - 0s 2ms/step - loss: 0.5296 - accuracy: 0.7877
Epoch 3/100
235/235 [==============================] - 0s 2ms/step - loss: 0.4640 - accuracy: 0.7953
Epoch 4/100
235/235 [==============================] - 0s 2ms/step - loss: 0.4343 - accuracy: 0.8047
Epoch 5/100
235/235 [==============================] - 0s 2ms/step - loss: 0.4174 - accuracy: 0.8124
Epoch 6/100
235/235 [==============================] - 0s 2ms/step - loss: 0.4044 - accuracy: 0.8240
Epoch 7/100
235/235 [==============================] - 0s 2ms/step - loss: 0.3931 - accuracy: 0.8332
Epoch 8/100
235/235 [==============================] - 0s 2ms/step - loss: 0.3820 - accuracy: 0.8409
Epoch 9/100
235/235 [==============================] - 0s 2ms/step - loss: 0.3715 - accuracy: 0.8489
Epoch 10/100
235/235 [==============================] - 0s 2ms/step - loss: 0.3633 - accura

In [ ]:
# Make predictions

# Single value
print(ann.predict(sc.transform([[1, 0, 0, 600, 1, 40, 3, 60000, 2, 1, 1, 50000]])) > 0.5)

# sc.transform is scaling the given value (which is what we did for the trained dataset) and the threshold is 0.5
# > 0.5 will return true, false otherwise

'''
country: France
credit score: 600
gender: male
age: 42
tenure:3
balance: 60000
number of products: 2
has credit card: 1
active member: 1
estimated salary: 50000
'''

1/1 [==============================] - 0s 228ms/step
[[False]]


In [ ]:
# Use values of test data and concatenate predicted and actual values

predictions = ann.predict(x_test)
predictions = (predictions > 0.5)

print(np.concatenate((predictions.reshape(len(predictions),1), y_test.reshape(len(y_test),1)),1))

79/79 [==============================] - 0s 1ms/step
[[0 1]
 [0 0]
 [0 1]
 ...
 [0 0]
 [1 1]
 [0 1]]


In [ ]:
# Model Accuracy Score
from sklearn.metrics import accuracy_score

accuracy_score(y_test, predictions)

0.8556